In [1]:
import pandas as pd
import pyodbc
conn = pyodbc.connect('Driver={SQL Server};'
                      'Server=kcplsql;'
                      'Database=KCPL_TBS_Archive;'
                      'Trusted_Connection=yes;')

computer = pd.read_sql_query('SELECT * FROM MyPC3SessionAudit WHERE StartTime > ?', conn, parse_dates=['StartTime'], params=['2022-01-01 20:26:15'])
computer.to_csv('data/computer.csv')
computer.head(2)

c:\Users\mohammedahmed\Anaconda3\envs\librarydata\lib\site-packages\pandas\io\sql.py:758: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,SessionID,StartTime,EndTime,SessionUserID,SessionResourceID,UserID,LoginName,FirstName,LastName,Type,...,LocationPolicyName,LocationSiteID,SiteName,SiteDescription,SitePolicyID,SitePolicyName,DesktopProfileID,DesktopProfileName,UserName,DateAdded
0,427608,2022-01-03 13:36:35,2022-01-03 13:44:43,58948,167,58948,1000129895412,"Hotchkiss, Samuel James",,1,...,Librarian Assigned,2,Central Library,"14 W. 10th Street, Kansas City, MO",2,General,3,KCPL Public,tbsuser,2022-01-03 23:45:05.487
1,427544,2022-01-03 12:22:40,2022-01-03 12:43:49,57564,187,57564,1000130260168,"Cowan, Calvin",,1,...,General,3,Lucile H. Bluford Branch,"3050 Prospect Avenue, Kansas City, MO",2,General,3,KCPL Public,tbsuser,2022-01-03 23:45:05.487


In [2]:
import pandas as pd 
computer = pd.read_csv('data/computer.csv', parse_dates=['StartTime'])
circulationjcb = pd.read_csv('data/Chromebooks stats.csv', parse_dates=['Trans Stat Date'])
circulationjcb_2 = circulationjcb.groupby(['Trans Stat Station Library','Trans Stat Itype', pd.Grouper(key='Trans Stat Date', freq='W-SAT',label='left',closed='left')])['Trans Stat Id'].count().reset_index()

circulationjan = pd.read_csv('data/M.Ahmed Charge Hist log output with IDs.csv', parse_dates=['Trans Hist Date'])
circulationaug = pd.read_csv('data/M.Ahmed Charge Hist log output with IDs8.csv', parse_dates=['Trans Hist Date'])
circulation = circulationjan.append(circulationaug, ignore_index=True).drop_duplicates()

# computer data
mask = computer['SiteName'].isin(['__Not In Use', '_default', '_IS Testing Lab'])
computer = computer[~mask]
replace_values = {'Lucile H. Bluford Branch': 'KC-BLUFORD', 'Plaza Branch': 'KC-PLAZA', 'Central Library': 'KC-CENTRAL', 'Waldo Branch': 'KC-WALDO', 'Southeast Branch': 'KC-SE', 'North-East Branch': 'KC-NE', 'Trails West Branch': 'KC-TRAILS', 'Westport Branch': 'KC-WSTPORT', 'Irene H. Ruiz Biblioteca de las Americas': 'KC-RUIZ', 'Sugar Creek Branch': 'KC-SGCREEK'}
computer = computer.replace({"SiteName": replace_values})
computer1 = computer[['StartTime', 'EndTime', 'SessionID', 'SiteName']]
computer1['Date'] = computer1['StartTime'].dt.date
computer1['time'] = computer1['StartTime'].dt.time
computer1['year'] = computer1['StartTime'].dt.year
computer1['week_day'] = computer1['StartTime'].dt.day_name()
computer1['time'] = computer1['time'].astype(str)
computer1['hour'] = computer1['time'].str.split(':').str[0]
computer1['hour'] = computer1['hour'].astype(int)
computer1['date'] = pd.to_datetime(computer1['Date'])
grouped = computer1.groupby(['date','SiteName', 'hour'])['SessionID'].count().reset_index()
replace_values = {8: '8 AM', 9: "9 AM", 10: "10 AM", 11: "11 AM", 12: "12 PM", 13: "1 PM", 14: "2 PM", 15: "3 PM", 16: "4 PM", 17: "5 PM", 18: "6 PM", 19: "7 PM", 20: "8 PM", 21: '9'}
computer_df = grouped.replace({"hour": replace_values})
computer_df.rename(columns={'date': 'Trans Hist Date', 'SiteName': 'Station Library Checkout', 'hour': 'hours'}, inplace=True)
mask1 = circulation['User Profile'].isin(['MISSING', 'KC-DISPLAY', 'KC-MAINT', 'DISCARD', 'KC-STAFF', 'DAMAGED', 'KC-CATALOG', 'KC-SUSPND',
 'LOST', 'KCP-ILL', 'KC-COLDEV', 'KC-TFRBTG', 'KC_CAT1', 'REPLACE'])
circulation = circulation[~mask1]
circulation['trans_time'] = circulation['Trans Hist Datetime'].str.split(' ').str[1]
circulation['hour'] = circulation['trans_time'].str.split(':').str[0]
circulation1 = circulation.groupby(['Trans Hist Date', 'Station Library Checkout', 'hour'])['User Id'].unique().reset_index()
## circulation1 = pd.DataFrame(circulation1)
#circulation1['patron'] = circulation['User Id'].unique()
circulation1['patrons'] = circulation1['User Id'].apply(lambda x: len(x))#.reset_index()
circulation2 = circulation1[['Trans Hist Date', 'Station Library Checkout', 'hour', 'patrons']]
#circulation2['patrons'] = circulation2['patrons'].fillna(0)
circulation2['hours'] = circulation2['hour'].astype(int)
circulation2['week_day'] = circulation2['Trans Hist Date'].dt.day_name()
replace_values = {9: "9 AM", 10: "10 AM", 11: "11 AM", 12: "12 PM", 13: "1 PM", 14: "2 PM", 15: "3 PM", 16: "4 PM", 17: "5 PM", 18: "6 PM", 19: "7 PM", 20: "8 PM"}
circulation2 = circulation2.replace({"hours": replace_values})
mer_df = pd.merge(circulation2, computer_df)
mer_df['total'] = mer_df['SessionID'] + mer_df['patrons']

<ipython-input-2-b5121ae46edc>:2: DtypeWarning: Columns (12) have mixed types. Specify dtype option on import or set low_memory=False.
  computer = pd.read_csv('data/computer.csv', parse_dates=['StartTime'])
<ipython-input-2-b5121ae46edc>:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  circulation = circulationjan.append(circulationaug, ignore_index=True).drop_duplicates()
<ipython-input-2-b5121ae46edc>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  computer1['Date'] = computer1['StartTime'].dt.date
<ipython-input-2-b5121ae46edc>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_

In [3]:
mer_df.head()

,Trans Hist Date,Station Library Checkout,hour,patrons,hours,week_day,SessionID,total
0,2022-01-03,KC-BLUFORD,10,1,10 AM,Monday,8,9
1,2022-01-03,KC-BLUFORD,11,1,11 AM,Monday,13,14
2,2022-01-03,KC-BLUFORD,12,1,12 PM,Monday,18,19
3,2022-01-03,KC-BLUFORD,13,1,1 PM,Monday,8,9
4,2022-01-03,KC-BLUFORD,14,2,2 PM,Monday,12,14


In [27]:
central  = mer_df[(mer_df['Station Library Checkout']=="KC-CENTRAL") & (mer_df['Trans Hist Date'] >= '2022-10-01') & (mer_df['Trans Hist Date'] <= '2022-10-31')]
central = central.groupby(['week_day','Trans Hist Date'])['total'].sum().reset_index()
#central = central.groupby(['week_day'])['total'].mean().reset_index()
central#.to_csv('Courtney_Grant_breakdown.csv')#.head()

,week_day,Trans Hist Date,total
0,Friday,2022-10-07,184
1,Friday,2022-10-14,178
2,Friday,2022-10-21,156
3,Friday,2022-10-28,182
4,Monday,2022-10-03,201
5,Monday,2022-10-10,192
6,Monday,2022-10-17,196
7,Monday,2022-10-24,199
8,Monday,2022-10-31,226
9,Saturday,2022-10-01,190


In [26]:
weekday = central[(central['week_day']=='Saturday') | (central['week_day']=='Sunday')]
weekday.mean()

<ipython-input-26-0fbf28ff0689>:2: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  weekday.mean()


total    135.4
dtype: float64

In [10]:
computers = computer.groupby(['SiteName'])['SessionResourceID'].unique().reset_index()
computers['Computers'] = computers['SessionResourceID'].str.len()
lst = computers['SessionResourceID'].str.len()
computers.to_csv('Computers for Ptron use by Branch.csv')
#lst

In [3]:
chromebookdf = pd.read_csv('data/Chromebooks stats.csv')
chromebookdf.head()

,Trans Stat Date,Trans Stat Itype,Trans Stat Station Library,Trans Stat User Profile Name,Trans Stat Id
0,2022-02-25,EQ-ECFHOT,KC-NE,KC-PUBLIC,20220225115646007580JZ2018297834
1,2022-02-27,EQ-ECFHOT,KC-BLUFORD,KC-PUBLIC,20220227155821006664FZ-1271751475
2,2022-02-27,EQ-ECFHOT,KC-NE,KC-PUBLIC,20220227153713006205JZ210224773
3,2022-02-28,EQ-ECFHOT,KC-CENTRAL,KC-PUBLIC,20220228204247012032JZ-1487371545
4,2022-02-28,EQ-ECFHOT,KC-NE,KC-PUBLIC,20220228123618008249JZ359197516


In [5]:
ch = chromebookdf[chromebookdf['Trans Stat Itype'] == 'EQ-ECFCB']
hs= chromebookdf[chromebookdf['Trans Stat Itype'] == 'EQ-ECFHOT']
print(ch['Trans Stat Id'].count())

2305


In [7]:
daily = ch.groupby(['Trans Stat Date'])['Trans Stat Id'].count().mean()
daily

17.20149253731343

In [4]:

circulationjcb_3 = chromebookdf.groupby(['Trans Stat Station Library','Trans Stat Date', 'Trans Stat Itype'])['Trans Stat Id'].count().reset_index()
both = circulationjcb_3[circulationjcb_3['Trans Stat Station Library'] == 'KC-BLUFORD'][['Trans Stat Itype','Trans Stat Station Library', 'Trans Stat Date', 'Trans Stat Id']]#.reset_index()
chromebook = both[both['Trans Stat Itype'] == 'EQ-ECFCB']
hotspot = both[both['Trans Stat Itype'] == 'EQ-ECFHOT']
hotspot

,Trans Stat Itype,Trans Stat Station Library,Trans Stat Date,Trans Stat Id
0,EQ-ECFHOT,KC-BLUFORD,2022-02-27,1
2,EQ-ECFHOT,KC-BLUFORD,2022-03-31,1
3,EQ-ECFHOT,KC-BLUFORD,2022-04-07,4
4,EQ-ECFHOT,KC-BLUFORD,2022-04-09,1
5,EQ-ECFHOT,KC-BLUFORD,2022-04-11,2
...,...,...,...,...
193,EQ-ECFHOT,KC-BLUFORD,2022-09-13,2
195,EQ-ECFHOT,KC-BLUFORD,2022-09-14,4
197,EQ-ECFHOT,KC-BLUFORD,2022-09-15,5
199,EQ-ECFHOT,KC-BLUFORD,2022-09-16,2


In [33]:
computer['StartTime'].max(0)

Timestamp('2022-09-06 20:36:25')

In [34]:
circulationjcb = pd.read_csv('data/M.Ahmed Charge Hist log output with IDscb.csv', parse_dates=['Trans Hist Date'])
circulationjcb = circulationjcb[circulationjcb['Item Type Code'] == 'EQ-ECFCB']
circulationjcb = circulationjcb.groupby(['Trans Hist Date', 'Station Library Checkout'])['Item Type Code'].count().reset_index()
circulationjcb.head()

,Trans Hist Date,Station Library Checkout,Item Type Code
0,2022-08-01,KC-BLUFORD,6
1,2022-08-01,KC-CENTRAL,13
2,2022-08-01,KC-NE,2
3,2022-08-01,KC-SE,3
4,2022-08-01,KC-TRAILS,2


In [35]:
chromebook = circulationjcb[circulationjcb['Station Library Checkout', 'Trans Hist Date'] == 'KC-RUIZ']['Item Type Code'].reset_index()
#len(chromebook)
chromebook#.to_csv('data/ruiz1.csv')

KeyError: ('Station Library Checkout', 'Trans Hist Date')

,Station Library Checkout,Trans Hist Date,Item Type Code
0,KC-BLUFORD,2022-03-26,2
1,KC-BLUFORD,2022-07-05,3
2,KC-BLUFORD,2022-07-07,3
3,KC-BLUFORD,2022-07-08,1
4,KC-BLUFORD,2022-07-11,4
...,...,...,...
132,KC-WSTPORT,2022-07-08,2
133,KC-WSTPORT,2022-07-09,5
134,KC-WSTPORT,2022-07-11,1
135,KC-WSTPORT,2022-07-12,3


In [ ]:
x=circulationjcb['Trans Hist Date'].reset_index()
x['month'] = x['Trans Hist Date'].dt.year
x['month'].unique()


array([2022], dtype=int64)

In [ ]:
added = circulationjcb.groupby(['Station Library Checkout', pd.Grouper(key='Trans Hist Date', freq='1Y')])['Item Type Code'].sum().reset_index()
added.head()

,Station Library Checkout,Trans Hist Date,Item Type Code
0,KC-BLUFORD,2022-12-31,24
1,KC-CENTRAL,2022-12-31,405
2,KC-MBL01,2022-12-31,3
3,KC-NE,2022-12-31,52
4,KC-OUTRCH,2022-12-31,3


In [ ]:
# cm =circulationjcb['Item Type Code'].value_counts().reset_index()
# cm.to_csv('data/Item Type Code.csv')

In [ ]:
circulationjan = pd.read_csv('data/M.Ahmed Charge Hist log output with IDs.csv', parse_dates=['Trans Hist Date'])
circulationaug = pd.read_csv('data/M.Ahmed Charge Hist log output with IDs8.csv', parse_dates=['Trans Hist Date'])
circulation = circulationjan.append(circulationaug, ignore_index=True).drop_duplicates()
circulation.head(1)
#circulationaug.head(1)

<ipython-input-3-d16f040ec8fa>:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  circulation = circulationjan.append(circulationaug, ignore_index=True).drop_duplicates()


,Trans Hist Id,Item Id,User Id,Trans Hist Date,Trans Hist Datetime,Item Barcode,User Barcode,User Profile,User Library,Station Library Checkout
0,20220103085739008612CV1867064041,2752762|1|25,895062,2022-01-03,1/3/2022 8:57,184680882,1.0001E+12,KC-OUTBTG,KC-OUTRCH,KC-OUTRCH


In [ ]:
import pandas as pd

computer = pd.read_csv('data/computer.csv', parse_dates=['StartTime'])
circulationjan = pd.read_csv('data/M.Ahmed Charge Hist log output with IDs.csv', parse_dates=['Trans Hist Date'])
circulationaug = pd.read_csv('data/M.Ahmed Charge Hist log output with IDs8.csv', parse_dates=['Trans Hist Date'])
circulation = circulationjan.append(circulationaug, ignore_index=True).drop_duplicates()

# computer data
mask = computer['SiteName'].isin(['__Not In Use', '_default', '_IS Testing Lab'])
computer = computer[~mask]
replace_values = {'Lucile H. Bluford Branch': 'KC-BLUFORD', 'Plaza Branch': 'KC-PLAZA', 'Central Library': 'KC-CENTRAL', 'Waldo Branch': 'KC-WALDO', 'Southeast Branch': 'KC-SE', 'North-East Branch': 'KC-NE', 'Trails West Branch': 'KC-TRAILS', 'Westport Branch': 'KC-WSTPORT', 'Irene H. Ruiz Biblioteca de las Americas': 'KC-RUIZ', 'Sugar Creek Branch': 'KC-SGCREEK'}
computer = computer.replace({"SiteName": replace_values})
computer1 = computer[['StartTime', 'EndTime', 'SessionID', 'SiteName']]
computer1['Date'] = computer1['StartTime'].dt.date
computer1['time'] = computer1['StartTime'].dt.time
computer1['year'] = computer1['StartTime'].dt.year
computer1['week_day'] = computer1['StartTime'].dt.day_name()
computer1['time'] = computer1['time'].astype(str)
computer1['hour'] = computer1['time'].str.split(':').str[0]
computer1['hour'] = computer1['hour'].astype(int)
computer1['date'] = pd.to_datetime(computer1['Date'])
grouped = computer1.groupby(['date','SiteName', 'hour'])['SessionID'].count().reset_index()
replace_values = {8: '8 AM', 9: "9 AM", 10: "10 AM", 11: "11 AM", 12: "12 PM", 13: "1 PM", 14: "2 PM", 15: "3 PM", 16: "4 PM", 17: "5 PM", 18: "6 PM", 19: "7 PM", 20: "8 PM", 21: '9'}
computer_df = grouped.replace({"hour": replace_values})
computer_df.rename(columns={'date': 'Trans Hist Date', 'SiteName': 'Station Library Checkout', 'hour': 'hours'}, inplace=True)
mask1 = circulation['User Profile'].isin(['MISSING', 'KC-DISPLAY', 'KC-MAINT', 'DISCARD', 'KC-STAFF', 'DAMAGED', 'KC-CATALOG', 'KC-SUSPND',
 'LOST', 'KCP-ILL', 'KC-COLDEV', 'KC-TFRBTG', 'KC_CAT1', 'REPLACE'])
circulation = circulation[~mask1]
circulation['trans_time'] = circulation['Trans Hist Datetime'].str.split(' ').str[1]
circulation['hour'] = circulation['trans_time'].str.split(':').str[0]
circulation1 = circulation.groupby(['Trans Hist Date', 'Station Library Checkout', 'hour'])['User Id'].unique().reset_index()
## circulation1 = pd.DataFrame(circulation1)
#circulation1['patron'] = circulation['User Id'].unique()
circulation1['patrons'] = circulation1['User Id'].apply(lambda x: len(x))#.reset_index()
circulation2 = circulation1[['Trans Hist Date', 'Station Library Checkout', 'hour', 'patrons']]
#circulation2['patrons'] = circulation2['patrons'].fillna(0)
circulation2['hours'] = circulation2['hour'].astype(int)
circulation2['week_day'] = circulation2['Trans Hist Date'].dt.day_name()
replace_values = {9: "9 AM", 10: "10 AM", 11: "11 AM", 12: "12 PM", 13: "1 PM", 14: "2 PM", 15: "3 PM", 16: "4 PM", 17: "5 PM", 18: "6 PM", 19: "7 PM", 20: "8 PM"}
circulation2 = circulation2.replace({"hours": replace_values})
mer_df = pd.merge(circulation2, computer_df)
mer_df['total'] = mer_df['SessionID'] + mer_df['patrons']
mer_df = pd.merge(mer_df, circulationjcb)

<ipython-input-2-b6fb9498dad8>:3: DtypeWarning: Columns (12) have mixed types. Specify dtype option on import or set low_memory=False.
  computer = pd.read_csv('data/computer.csv', parse_dates=['StartTime'])
<ipython-input-2-b6fb9498dad8>:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  circulation = circulationjan.append(circulationaug, ignore_index=True).drop_duplicates()
<ipython-input-2-b6fb9498dad8>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  computer1['Date'] = computer1['StartTime'].dt.date
<ipython-input-2-b6fb9498dad8>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_

NameError: name 'circulationjcb' is not defined

In [ ]:
circulationjcb = pd.read_csv('data/M.Ahmed Charge Hist log output with IDscb.csv', parse_dates=['Trans Hist Date'])
circulationjcb = circulationjcb[circulationjcb['Item Type Code'] == 'EQ-ECFCB']
circulationjcb = circulationjcb.groupby(['Trans Hist Date', 'Station Library Checkout'])['Item Type Code'].count().reset_index()

circulationjan = pd.read_csv('data/M.Ahmed Charge Hist log output with IDs.csv', parse_dates=['Trans Hist Date'])
circulationaug = pd.read_csv('data/M.Ahmed Charge Hist log output with IDs8.csv', parse_dates=['Trans Hist Date'])
circulation = circulationjan.append(circulationaug, ignore_index=True).drop_duplicates()

#circulation = pd.read_csv('data/M.Ahmed Charge Hist log output with IDs.csv', parse_dates=['Trans Hist Date'])
# import pyodbc
# conn = pyodbc.connect('Driver={SQL Server};'
#                       'Server=kcplsql;'
#                       'Database=KCPL_TBS_Archive;'
#                       'Trusted_Connection=yes;')

# computer = pd.read_sql_query('SELECT * FROM MyPC3SessionAudit WHERE StartTime > ?', conn, parse_dates=['StartTime'], params=['2022-01-21 20:26:15'])
computer = pd.read_csv('data/computer.csv', parse_dates=['StartTime'])

# computer data
mask = computer['SiteName'].isin(['__Not In Use', '_default', '_IS Testing Lab'])
computer = computer[~mask]
replace_values = {'Lucile H. Bluford Branch': 'KC-BLUFORD', 'Plaza Branch': 'KC-PLAZA', 'Central Library': 'KC-CENTRAL', 'Waldo Branch': 'KC-WALDO', 'Southeast Branch': 'KC-SE', 'North-East Branch': 'KC-NE', 'Trails West Branch': 'KC-TRAILS', 'Westport Branch': 'KC-WSTPORT', 'Irene H. Ruiz Biblioteca de las Americas': 'KC-RUIZ', 'Sugar Creek Branch': 'KC-SGCREEK'}
computer = computer.replace({"SiteName": replace_values})
computer1 = computer[['StartTime', 'EndTime', 'SessionID', 'SiteName']]
computer1['Date'] = computer1['StartTime'].dt.date
computer1['time'] = computer1['StartTime'].dt.time
computer1['year'] = computer1['StartTime'].dt.year
computer1['week_day'] = computer1['StartTime'].dt.day_name()
computer1['time'] = computer1['time'].astype(str)
computer1['hour'] = computer1['time'].str.split(':').str[0]
computer1['hour'] = computer1['hour'].astype(int)
computer1['date'] = pd.to_datetime(computer1['Date'])
grouped = computer1.groupby(['date','SiteName', 'hour'])['SessionID'].count().reset_index()
replace_values = {8: '8 AM', 9: "9 AM", 10: "10 AM", 11: "11 AM", 12: "12 PM", 13: "1 PM", 14: "2 PM", 15: "3 PM", 16: "4 PM", 17: "5 PM", 18: "6 PM", 19: "7 PM", 20: "8 PM", 21: '9'}
computer_df = grouped.replace({"hour": replace_values})
computer_df.rename(columns={'date': 'Trans Hist Date', 'SiteName': 'Station Library Checkout', 'hour': 'hours'}, inplace=True)
mask1 = circulation['User Profile'].isin(['MISSING', 'KC-DISPLAY', 'KC-MAINT', 'DISCARD', 'KC-STAFF', 'DAMAGED', 'KC-CATALOG', 'KC-SUSPND',
 'LOST', 'KCP-ILL', 'KC-COLDEV', 'KC-TFRBTG', 'KC_CAT1', 'REPLACE'])
circulation = circulation[~mask1]
circulation['trans_time'] = circulation['Trans Hist Datetime'].str.split(' ').str[1]
circulation['hour'] = circulation['trans_time'].str.split(':').str[0]
circulation1 = circulation.groupby(['Trans Hist Date', 'Station Library Checkout', 'hour'])['User Id'].unique().reset_index()
## circulation1 = pd.DataFrame(circulation1)
#circulation1['patron'] = circulation['User Id'].unique()
circulation1['patrons'] = circulation1['User Id'].apply(lambda x: len(x))#.reset_index()
circulation2 = circulation1[['Trans Hist Date', 'Station Library Checkout', 'hour', 'patrons']]
#circulation2['patrons'] = circulation2['patrons'].fillna(0)
circulation2['hours'] = circulation2['hour'].astype(int)
circulation2['week_day'] = circulation2['Trans Hist Date'].dt.day_name()
replace_values = {9: "9 AM", 10: "10 AM", 11: "11 AM", 12: "12 PM", 13: "1 PM", 14: "2 PM", 15: "3 PM", 16: "4 PM", 17: "5 PM", 18: "6 PM", 19: "7 PM", 20: "8 PM"}
circulation2 = circulation2.replace({"hours": replace_values})
mer_df = pd.merge(circulation2, computer_df)
mer_df['total'] = mer_df['SessionID'] + mer_df['patrons']

<ipython-input-6-eab7dbde5c02>:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  circulation = circulationjan.append(circulationaug, ignore_index=True).drop_duplicates()
<ipython-input-6-eab7dbde5c02>:17: DtypeWarning: Columns (12) have mixed types. Specify dtype option on import or set low_memory=False.
  computer = pd.read_csv('data/computer.csv', parse_dates=['StartTime'])
<ipython-input-6-eab7dbde5c02>:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  computer1['Date'] = computer1['StartTime'].dt.date
<ipython-input-6-eab7dbde5c02>:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col

In [38]:
circulation['User Profile'].value_counts()
# 'KC-OUTBTG', 'KC-EDUC', 'KC-OUT-HBB', 'KC-ISMAINT', 'KC-TFROUT', 'KC-MONTH', 'HOTSWAP', 'ZCKCCENTRA', 'BKBAG-RCRC'

KC-PUBLIC     290820
KC-OUTBTG      48540
KC-NEWREG      28192
KC-EDUC         9434
KC-OUT-HBB      8979
KC-ONLINE       2533
SCH-ELEM        2336
SCH-HS          1238
KC-ISMAINT       623
KC-TFROUT        204
SCH-STAFF        115
LAF-STUDNT       104
KC-MONTH          74
HOTSWAP           62
ZCKCCENTRA         5
BKBAG-RCRC         1
Name: User Profile, dtype: int64

In [ ]:
covid_data_1w = mer_df.groupby([pd.Grouper(key='Trans Hist Date', freq='W-SAT',label='left',closed='left')])[['patrons', 'SessionID', 'total']].sum().reset_index()
#covid_data_1w
mer_df_2 = mer_df.groupby(['Station Library Checkout', pd.Grouper(key='Trans Hist Date', freq='W-SAT',label='left',closed='left')])[['patrons', 'SessionID', 'total']].sum().reset_index()
value_confirmed = mer_df_2[mer_df_2['Station Library Checkout'] == 'KC-PLAZA']['total'].iloc[-2]
delta_confirmed = mer_df_2[mer_df_2['Station Library Checkout'] == 'KC-PLAZA']['total'].iloc[-3] 
value_confirmed

1889

In [ ]:
delta_confirmed

1744

In [ ]:
computer_df.head()
#print(computer_df['Trans Hist Date'].min())

,Trans Hist Date,Station Library Checkout,hours,SessionID
0,2022-01-03,KC-BLUFORD,9 AM,6
1,2022-01-03,KC-BLUFORD,10 AM,8
2,2022-01-03,KC-BLUFORD,11 AM,13
3,2022-01-03,KC-BLUFORD,12 PM,18
4,2022-01-03,KC-BLUFORD,1 PM,8


In [ ]:
ne_computer = computer_df[computer_df['Station Library Checkout'] == 'KC-NE'].reset_index()
by_hour = ne_computer.groupby(['Trans Hist Date'])['SessionID'].sum().reset_index()
by_hour
by_hour.to_csv('data/NE_computerSessionsAGGbyDay.csv')

In [ ]:
# mer_df = pd.merge(mer_df, circulationjcb)
mer_df.head()

,Trans Hist Date,Station Library Checkout,hour,patrons,hours,week_day,SessionID,total,Item Type Code
0,2022-01-07,KC-CENTRAL,10,8,10 AM,Friday,5,13,1
1,2022-01-07,KC-CENTRAL,11,2,11 AM,Friday,11,13,1
2,2022-01-07,KC-CENTRAL,12,7,12 PM,Friday,7,14,1
3,2022-01-07,KC-CENTRAL,13,8,1 PM,Friday,4,12,1
4,2022-01-07,KC-CENTRAL,14,13,2 PM,Friday,5,18,1


In [ ]:
mer_df.head()#['Trans Hist Date'].max()

,Trans Hist Date,Station Library Checkout,hour,patrons,hours,week_day,SessionID,total
0,2022-01-03,KC-BLUFORD,10,1,10 AM,Monday,8,9
1,2022-01-03,KC-BLUFORD,11,1,11 AM,Monday,13,14
2,2022-01-03,KC-BLUFORD,12,1,12 PM,Monday,18,19
3,2022-01-03,KC-BLUFORD,13,1,1 PM,Monday,8,9
4,2022-01-03,KC-BLUFORD,14,2,2 PM,Monday,12,14


In [ ]:
#mer_df['Station Library Checkout'].unique

Ruiz traffic


In [ ]:
ruiz = mer_df[mer_df['Station Library Checkout'] == 'KC-RUIZ'].reset_index()
ruiz.head()

,index,Trans Hist Date,Station Library Checkout,hour,patrons,hours,week_day,SessionID,total
0,36,2022-01-03,KC-RUIZ,11,2,11 AM,Monday,2,4
1,170,2022-01-05,KC-RUIZ,12,2,12 PM,Wednesday,1,3
2,231,2022-01-06,KC-RUIZ,16,1,4 PM,Thursday,1,2
3,350,2022-01-08,KC-RUIZ,12,1,12 PM,Saturday,1,2
4,446,2022-01-10,KC-RUIZ,12,1,12 PM,Monday,1,2


In [ ]:
# max per day
rfcast= ruiz.groupby(['Trans Hist Date', 'week_day'])['total'].sum().reset_index()
rfcast.head()

,Trans Hist Date,week_day,total
0,2022-01-03,Monday,4
1,2022-01-05,Wednesday,3
2,2022-01-06,Thursday,2
3,2022-01-08,Saturday,2
4,2022-01-10,Monday,2


In [ ]:
# ruiz first hour
ruiz1 = mer_df[(mer_df['Station Library Checkout'] == 'KC-RUIZ') & (mer_df['hours'] == '11 AM')].reset_index()
ruiz1.head()

,index,Trans Hist Date,Station Library Checkout,hour,patrons,hours,week_day,SessionID,total
0,36,2022-01-03,KC-RUIZ,11,2,11 AM,Monday,2,4
1,515,2022-01-11,KC-RUIZ,11,1,11 AM,Tuesday,2,3
2,822,2022-01-18,KC-RUIZ,11,1,11 AM,Tuesday,3,4
3,1041,2022-01-21,KC-RUIZ,11,1,11 AM,Friday,4,5
4,1100,2022-01-22,KC-RUIZ,11,1,11 AM,Saturday,2,3


In [ ]:
# ruiz m-th 6 PM
ruiz_m_th = mer_df[(mer_df['Station Library Checkout'] == 'KC-RUIZ') & (mer_df['hours'] == '11 AM')].reset_index()
ruiz_m_th.head()

,index,Trans Hist Date,Station Library Checkout,hour,patrons,hours,week_day,SessionID,total
0,36,2022-01-03,KC-RUIZ,11,2,11 AM,Monday,2,4
1,515,2022-01-11,KC-RUIZ,11,1,11 AM,Tuesday,2,3
2,822,2022-01-18,KC-RUIZ,11,1,11 AM,Tuesday,3,4
3,1041,2022-01-21,KC-RUIZ,11,1,11 AM,Friday,4,5
4,1100,2022-01-22,KC-RUIZ,11,1,11 AM,Saturday,2,3


In [ ]:
import numpy as np
days_chart = mer_df.groupby(['Station Library Checkout','Trans Hist Date', 'hours'])['total'].sum().reset_index()
days_chart = days_chart[days_chart['Station Library Checkout'] == "KC-RUIZ"][['Station Library Checkout', 'Trans Hist Date','hours', 'total']].reset_index()
days_chart = pd.pivot_table(days_chart, columns='hours', values='total', index='Trans Hist Date').fillna(0).round(decimals = 0)#apply(np.ceil)
days_chart.head(2)#.to_csv('data/daysnum.csv')

hours,1 PM,10 AM,11 AM,12 PM,2 PM,3 PM,4 PM,5 PM,6 PM
Trans Hist Date,,,,,,,,,
2022-01-03,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0
2022-01-05,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
days_chart.index[0]

Timestamp('2022-01-03 00:00:00')

In [ ]:
circulationjcb_2 = circulationjcb.groupby(['Station Library Checkout', pd.Grouper(key='Trans Hist Date', freq='1D')])['Item Type Code'].sum().reset_index()
value_confirmed1 = circulationjcb_2[circulationjcb_2['Station Library Checkout'] == 'KC-RUIZ']['Item Type Code'].iloc[-1]
delta_confirmed1 = circulationjcb_2[circulationjcb_2['Station Library Checkout'] == 'KC-RUIZ']['Item Type Code'].iloc[-2]
circulationjcb_2.head()

,Station Library Checkout,Trans Hist Date,Item Type Code
0,KC-BLUFORD,2022-03-26,2
1,KC-BLUFORD,2022-07-05,3
2,KC-BLUFORD,2022-07-07,3
3,KC-BLUFORD,2022-07-08,1
4,KC-BLUFORD,2022-07-11,4


In [ ]:
cols = {'Trans Hist Date':'2022-01-03 00:00:00','Station Library Checkout':['KC-RUIZ'], 'Item Type Code':[0]}

empty = pd.DataFrame(data=cols)#.set_index("")
circulationjcb_2 = empty.append(circulationjcb_2, ignore_index=False).drop_duplicates().reset_index().fillna(0)
circulationjcb_2


<ipython-input-43-8d82c35ab4e8>:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  circulationjcb_2 = empty.append(circulationjcb_2, ignore_index=False).drop_duplicates().reset_index().fillna(0)


,index,Trans Hist Date,Station Library Checkout,Item Type Code
0,0,2022-01-03 00:00:00,KC-RUIZ,0
1,0,2022-03-26 00:00:00,KC-BLUFORD,2
2,1,2022-07-05 00:00:00,KC-BLUFORD,3
3,2,2022-07-07 00:00:00,KC-BLUFORD,3
4,3,2022-07-08 00:00:00,KC-BLUFORD,1
...,...,...,...,...
133,132,2022-07-08 00:00:00,KC-WSTPORT,2
134,133,2022-07-09 00:00:00,KC-WSTPORT,5
135,134,2022-07-11 00:00:00,KC-WSTPORT,1
136,135,2022-07-12 00:00:00,KC-WSTPORT,3


In [ ]:
mer = empty.append(days_chart, ignore_index=False).drop_duplicates().reset_index().fillna(0)
mer

<ipython-input-78-4975c0cb4f3d>:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  mer = empty.append(days_chart, ignore_index=False).drop_duplicates().reset_index().fillna(0)
<ipython-input-78-4975c0cb4f3d>:1: FutureWarning: Inferring datetime64[ns] from data containing strings is deprecated and will be removed in a future version. To retain the old behavior explicitly pass Series(data, dtype=datetime64[ns])
  mer = empty.append(days_chart, ignore_index=False).drop_duplicates().reset_index().fillna(0)


,index,1 PM,10 AM,11 AM,12 PM,2 PM,3 PM,4 PM,5 PM,6 PM,7 PM,8 PM,9 AM
0,2022-01-03,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2022-01-03,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2022-01-05,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2022-01-06,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0
4,2022-01-08,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
145,2022-07-26,4.0,0.0,0.0,4.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0
146,2022-07-27,3.0,0.0,8.0,5.0,0.0,0.0,0.0,6.0,0.0,0.0,0.0,0.0
147,2022-07-28,0.0,0.0,2.0,3.0,3.0,0.0,0.0,4.0,5.0,0.0,0.0,0.0
148,2022-07-29,0.0,0.0,5.0,0.0,2.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
days_chart.columns

Index(['1 PM', '10 AM', '11 AM', '12 PM', '2 PM', '3 PM', '4 PM', '5 PM',
       '6 PM', '7 PM', '8 PM', '9 AM'],
      dtype='object', name='hours')

In [ ]:
# RUIZ M-TH
r_m_th = ruiz_m_th.groupby(['Trans Hist Date', 'week_day'])['total'].sum().reset_index()
r_m_thp = pd.pivot_table(r_m_th, columns='week_day', values='total', index='Trans Hist Date').fillna(0).astype(int)
r_m_thp#.head()
#r_m_thp.to_csv('ruiz-wesport/RUIZ_11Am.csv')

week_day,Friday,Monday,Saturday,Thursday,Tuesday,Wednesday
Trans Hist Date,,,,,,
2022-01-03,0,4,0,0,0,0
2022-01-11,0,0,0,0,3,0
2022-01-18,0,0,0,0,4,0
2022-01-21,5,0,0,0,0,0
2022-01-22,0,0,3,0,0,0
...,...,...,...,...,...,...
2022-07-21,0,0,0,2,0,0
2022-07-23,0,0,3,0,0,0
2022-07-27,0,0,0,0,0,8


In [ ]:
# # ruiz first hour
# ruiz1 = mer_df[mer_df['Station Library Checkout'] == 'KC-RUIZ'].reset_index()
# ruiz1.head()

In [ ]:
#$ruiz['hour'].min()
d_avg = ruiz.groupby(['hours', 'week_day'])['total'].sum().reset_index()
d_avgp = pd.pivot_table(d_avg, columns='week_day', values='total', index='hours').fillna(0).astype(int)
d_avgp.head()

week_day,Friday,Monday,Saturday,Thursday,Tuesday,Wednesday
hours,,,,,,
1 PM,40,24,29,21,18,25
10 AM,0,0,44,0,0,0
11 AM,60,45,72,48,78,44
12 PM,27,28,79,51,35,33
2 PM,24,23,0,19,33,47


In [ ]:
d_avgp.describe()

week_day,Friday,Monday,Saturday,Thursday,Tuesday,Wednesday
count,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000
mean,26.666667,27.222222,24.888889,30.111111,43.333333,37.777778
std,22.655022,14.350765,32.838409,17.164239,25.792441,21.446315
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,23.000000,0.000000,20.000000,33.000000,27.000000
50%,27.000000,27.000000,0.000000,28.000000,37.000000,40.000000
75%,40.000000,40.000000,44.000000,48.000000,67.000000,44.000000
max,60.000000,45.000000,79.000000,51.000000,78.000000,80.000000


In [ ]:
# Ruiz Daily Traffic
daily = ruiz1.groupby(['Trans Hist Date', 'week_day'])['total'].sum().reset_index()
dailyp = pd.pivot_table(daily, columns='week_day', values='total', index='Trans Hist Date').mean().reset_index()#.fillna(0).astype(int)
dailyp#.head()

,week_day,0
0,Friday,6.000000
1,Monday,4.090909
2,Saturday,4.000000
3,Thursday,4.000000
4,Tuesday,6.500000
5,Wednesday,4.400000


In [ ]:
avg = (1208 + 1030 + 1070 + 1122 + 1249)/5
print(avg)

1135.8


In [ ]:
# ruizp = dailyp[['10 AM', '11 AM', '12 PM', '1 PM', '2 PM', '3 PM', '4 PM', '5 PM', '6 PM']]
# #ruizp.to_csv('ruiz-wesport/ruiz_traffic.csv')

In [ ]:
# # Set x axis to numerical value for month
# import matplotlib.pyplot as plt
# import numpy as np
# x_axis = np.arange(1,118,1)
# plt.figure(figsize=(20,6))
# ten, = plt.plot(ruizp.index, ruizp['10 AM'], color="g", linewidth=1, label="10 AM")
# elevent, = plt.plot(ruizp.index, ruizp['11 AM'], color="b", linewidth=1, label="11 AM")
# noon, = plt.plot(ruizp.index, ruizp["12 PM"], color="c", linewidth=1, label="12 PM")
# one, = plt.plot(ruizp.index, ruizp['1 PM'], color="r", linewidth=1, label="1 PM")
# two, = plt.plot(ruizp.index, ruizp['2 PM'], color="m", linewidth=1, label="2 PM")
# three, = plt.plot(ruizp.index, ruizp['3 PM'], color="y", linewidth=1, label="3 PM")
# four, = plt.plot(ruizp.index, ruizp["4 PM"], color="k", linewidth=1, label="4 PM")
# five, = plt.plot(ruizp.index, ruizp['5 PM'], color="m", linewidth=1, label="5 PM", linestyle='dashdot')
# six, = plt.plot(ruizp.index, ruizp['6 PM'], color="r", linewidth=1, label="6 PM", linestyle='dashed')
# plt.legend(handles=[elevent, two, four, three, ten, noon, one, six, five], loc="best")


# plt.title("Weekdays patrons traffic by Branch and day of the Month (Jul 2021 - Jan 2022)")
# plt.xlabel("day of the month")
# plt.ylabel("Number of patrons")
# plt.xticks(rotation=90)
# # Save and display the chart
# #plt.ylim(0, 350)
# #plt.savefig("img/incidents_per_month.png")
# plt.show()




Westport Traffic

In [ ]:
westport = mer_df[mer_df['Station Library Checkout'] == 'KC-WSTPORT'].reset_index()
westport.head()

,index,Trans Hist Date,Station Library Checkout,hour,patrons,hours,week_day,SessionID,total
0,62,2022-01-03,KC-WSTPORT,10,2,10 AM,Monday,7,9
1,63,2022-01-03,KC-WSTPORT,11,5,11 AM,Monday,1,6
2,64,2022-01-03,KC-WSTPORT,12,7,12 PM,Monday,3,10
3,65,2022-01-03,KC-WSTPORT,13,9,1 PM,Monday,4,13
4,66,2022-01-03,KC-WSTPORT,14,4,2 PM,Monday,4,8


In [ ]:
# ruiz first hour
west1 = mer_df[(mer_df['Station Library Checkout'] == 'KC-WSTPORT') & (mer_df['hours'] == '4 PM')].reset_index()
west1.head()

,index,Trans Hist Date,Station Library Checkout,hour,patrons,hours,week_day,SessionID,total
0,68,2022-01-03,KC-WSTPORT,16,1,4 PM,Monday,2,3
1,133,2022-01-04,KC-WSTPORT,16,5,4 PM,Tuesday,6,11
2,197,2022-01-05,KC-WSTPORT,16,5,4 PM,Wednesday,3,8
3,262,2022-01-06,KC-WSTPORT,16,2,4 PM,Thursday,5,7
4,322,2022-01-07,KC-WSTPORT,16,2,4 PM,Friday,3,5


In [ ]:
# Ruiz Daily Traffic
daily1 = west1.groupby(['Trans Hist Date', 'week_day'])['total'].sum().reset_index()
dailyp1 = pd.pivot_table(daily1, columns='week_day', values='total', index='Trans Hist Date').mean().reset_index()#.fillna(0).astype(int)
dailyp1#.head()

,week_day,0
0,Friday,6.640000
1,Monday,6.458333
2,Saturday,5.066667
3,Thursday,6.434783
4,Tuesday,7.689655
5,Wednesday,7.241379


In [ ]:

d_avg1 = westport.groupby(['hours', 'week_day'])['total'].sum().reset_index()
d_avgp1 = pd.pivot_table(d_avg1, columns='week_day', values='total', index='hours').fillna(0).astype(int)#.mean().reset_index()
d_avgp1#.to_csv('ruiz-wesport/wesport_traffic.csv')#.head()

week_day,Friday,Monday,Saturday,Thursday,Tuesday,Wednesday
hours,,,,,,
1 PM,165,160,156,159,189,175
10 AM,202,200,161,193,212,206
11 AM,166,141,165,135,184,189
12 PM,198,155,184,141,170,169
2 PM,205,156,158,178,208,221
3 PM,224,182,149,162,201,192
4 PM,166,155,76,148,223,210
5 PM,0,62,0,117,87,150
6 PM,0,0,0,35,0,26


In [ ]:
d_avgp1.describe()

week_day,Friday,Monday,Saturday,Thursday,Tuesday,Wednesday
count,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000
mean,147.333333,134.555556,116.555556,140.888889,163.777778,170.888889
std,86.010174,63.134996,72.467426,45.736868,73.293550,58.626880
min,0.000000,0.000000,0.000000,35.000000,0.000000,26.000000
25%,165.000000,141.000000,76.000000,135.000000,170.000000,169.000000
50%,166.000000,155.000000,156.000000,148.000000,189.000000,189.000000
75%,202.000000,160.000000,161.000000,162.000000,208.000000,206.000000
max,224.000000,200.000000,184.000000,193.000000,223.000000,221.000000


In [ ]:
mer_df['Trans Hist Date'].max()

Timestamp('2022-07-31 00:00:00')

In [ ]:
#mer_df['Trans Hist Date'] = pd.to_datetime(mer_df['Trans Hist Date'])
jan_apr = mer_df[(mer_df['Trans Hist Date'] < '2022-05-01 00:00:00') & (mer_df['Station Library Checkout'] == 'KC-CENTRAL') & (mer_df['hour'] > '16')]
jan_apr['month'] = jan_apr['Trans Hist Date'].dt.month_name()

jan_apr

<ipython-input-24-c345d70a5a3b>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  jan_apr['month'] = jan_apr['Trans Hist Date'].dt.month_name()


,Trans Hist Date,Station Library Checkout,hour,patrons,hours,week_day,SessionID,total,month
15,2022-01-03,KC-CENTRAL,17,18,5 PM,Monday,2,20,January
16,2022-01-03,KC-CENTRAL,18,9,6 PM,Monday,3,12,January
17,2022-01-03,KC-CENTRAL,19,2,7 PM,Monday,3,5,January
18,2022-01-03,KC-CENTRAL,20,3,8 PM,Monday,1,4,January
19,2022-01-03,KC-CENTRAL,9,2,9 AM,Monday,9,11,January
...,...,...,...,...,...,...,...,...,...
6698,2022-04-27,KC-CENTRAL,20,3,8 PM,Wednesday,2,5,April
6699,2022-04-27,KC-CENTRAL,9,5,9 AM,Wednesday,17,22,April
6778,2022-04-28,KC-CENTRAL,9,5,9 AM,Thursday,18,23,April
6853,2022-04-29,KC-CENTRAL,9,5,9 AM,Friday,14,19,April


In [ ]:
grouped = jan_apr.groupby(['month', 'week_day','hours'])['total'].sum().reset_index()
grouped.head()#.to_csv('data/jan_april.csv')

,month,week_day,hours,total
0,April,Friday,9 AM,84
1,April,Monday,5 PM,56
2,April,Monday,6 PM,47
3,April,Monday,7 PM,31
4,April,Monday,8 PM,27


In [ ]:
grouped['week_day'].value_counts()

Monday       20
Tuesday      20
Wednesday    20
Friday        4
Saturday      4
Thursday      4
Name: week_day, dtype: int64

In [ ]:
pivoted = pd.pivot_table(grouped, columns='hours', values='total', index=['month','week_day'])
pivoted#.to_csv('data/allweekindex.csv')

hours               5 PM  6 PM  7 PM  8 PM  9 AM
month    week_day                               
April    Friday      NaN   NaN   NaN   NaN  84.0
         Monday     56.0  47.0  31.0  27.0  66.0
         Saturday    NaN   NaN   NaN   NaN  74.0
         Thursday    NaN   NaN   NaN   NaN  92.0
         Tuesday    42.0  34.0  27.0  13.0  67.0
         Wednesday  52.0  42.0  35.0  24.0  69.0
February Friday      NaN   NaN   NaN   NaN  46.0
         Monday     49.0  20.0  17.0   6.0  38.0
         Saturday    NaN   NaN   NaN   NaN  62.0
         Thursday    NaN   NaN   NaN   NaN  38.0
         Tuesday    66.0  33.0   9.0   2.0  71.0
         Wednesday  24.0  26.0  17.0   8.0  51.0
January  Friday      NaN   NaN   NaN   NaN  52.0
         Monday     63.0  46.0  26.0  16.0  54.0
         Saturday    NaN   NaN   NaN   NaN  28.0
         Thursday    NaN   NaN   NaN   NaN  47.0
         Tuesday    51.0  33.0  15.0   6.0  53.0
         Wednesday  54.0  26.0   7.0  17.0  46.0
March    Friday      NaN   NaN   NaN   NaN  50.0
         Monday     41.0  33.0  38.0  13.0  49.0
         Saturday    NaN   NaN   NaN   NaN  66.0
         Thursday    NaN   NaN   NaN   NaN  48.0
         Tuesday    55.0  36.0  37.0  18.0  66.0
         Wednesday  70.0  45.0  46.0  15.0  70.0